In [1]:
import pandas as pd
import numpy as np

In [5]:
ttc = pd.read_csv('ttc_delays_cleaned.csv')

In [4]:
stations = pd.read_csv('csv_originals/station_ridership_grade.csv')
stations

,Station,Grade,2015 ridership
0,College,Underground,"47,790"
1,Davisville,Surface,"25,330"
2,Dundas,Underground,"81,330"
3,Dupont,Underground,"16,140"
4,Eglinton,Underground,"72,750"
5,Eglinton West,Underground/Surface,"16,830"
6,Finch,Underground,"100,820"
7,Glencairn,Surface,"5,720"
8,King,Underground,"55,810"
9,Lawrence,Underground,"24,560"
